In [26]:
## Import libraries
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})  # enlarge matplotlib fonts
import pickle
import os

In [27]:
# Import qubit states Zero (|0>) and One (|1>), and Pauli operators (X, Y, Z)
from qiskit.opflow import Zero, One, I, X, Y, Z

In [28]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [29]:
## Import functions from Qiskit
from qiskit                     import QuantumCircuit, QuantumRegister, IBMQ, execute, transpile, Aer
from qiskit.providers.aer       import QasmSimulator
from qiskit.tools.monitor       import job_monitor
from qiskit.circuit             import Parameter, ParameterVector
from qiskit.quantum_info        import Statevector, Pauli
from qiskit.opflow.state_fns    import CircuitStateFn
from qiskit.opflow.expectations import PauliExpectation
from qiskit.utils               import QuantumInstance
from qiskit.opflow              import PauliOp, SummedOp, CircuitSampler, StateFn

In [30]:
# Import state tomography modules
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info                  import state_fidelity

In [31]:
## Mitiq libraries
from mitiq import zne
from qiskit.result import Result
from qiskit.result.models import ExperimentResult
from qiskit.result.models import ExperimentResultData
from qiskit.result.models import QobjExperimentHeader

In [32]:
from itertools import chain

## Create the circuit

In [33]:
def Heisenberg_YBE_variational(num_qubits,p):

    circ  = QuantumCircuit(num_qubits)
    count = 0
    
    def XYZ_variational(circ,i,j,params):
        circ.cx(i,j)
        circ.rx(params[0],i)
        circ.rx(-np.pi/2,i)
        circ.h(i)
        circ.rz(params[1],j)

        circ.cx(i,j)
        circ.h(i)
        circ.rz(params[2],j)

        circ.cx(i,j)
        circ.rx(np.pi/2,i)
        circ.rx(-np.pi/2,j)

    circ.rx(np.pi,[1,2])
    
    XYZ_variational(circ,1,2,p[count:count+3])
    count += 3
    XYZ_variational(circ,0,1,p[count:count+3])
    count += 3
    XYZ_variational(circ,1,2,p[count:count+3])
    count += 3
    XYZ_variational(circ,0,1,p[count:count+3])
    count += 3
    XYZ_variational(circ,1,2,p[count:count+3])
    count += 3

    return circ

In [34]:
pvqd_opt_params = [0.6382017062070897,
0.5999999987484098,
0.6382017062066773,
3.0088034895496003,
-3.0869200336945677,
0.4709531470409451,
2.163149581322057,
3.480816125849344,
-2.0741264452466974,
1.2330206913091548,
3.1275100711382064,
1.593744340473751,
6.107319841483039,
3.0177717815840808,
-3.24901805128811]

In [10]:
## Create the circuit
# Define the final circuit that is used to compute the fidelity 
fqr = QuantumRegister(7)
fqc = QuantumCircuit(fqr)
#fqc.rx(np.pi, [3, 5]) # Cannot use X gate due to a bug in mitq, rx(pi) does the same thing
fqc.id([0, 1, 2, 4, 6]) # Need to put identities since mitq cannot handle unused qubits
fqc.append(Heisenberg_YBE_variational(3,pvqd_opt_params), [fqr[1], fqr[3], fqr[5]])

## Run on hardware

In [36]:
## Info for IBM
#IBMQ.save_account('MY_API_TOKEN')
#IBMQ.enable_account('MY_API_TOKEN')
IBMQ.load_account()

ibmqfactory.load_account:WARNING:2022-03-25 15:48:25,946: Credentials are already in use. The existing account in the session will be replaced.


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [37]:
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')
#provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
jakarta = provider.get_backend('ibmq_jakarta')
#bogota   = provider.get_backend('ibmq_bogota')

# Simulated backend based on ibmq_jakarta's device noise profile
#sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))

In [38]:
shots = 8192
#backend = sim_noisy_jakarta
backend = jakarta
#backend = bogota

In [39]:
# Compute the state tomography based on the st_qcs quantum circuits and the results from those ciricuits
def state_tomo(result, st_qcs):
    # The expected final state; necessary to determine state tomography fidelity
    target_state = (One^One^Zero).to_matrix()  # DO NOT MODIFY (|q_5,q_3,q_1> = |110>)
    # Fit state tomography results
    tomo_fitter = StateTomographyFitter(result, st_qcs)
    rho_fit = tomo_fitter.fit(method='lstsq')
    # Compute fidelity
    fid = state_fidelity(rho_fit, target_state)
    return fid

### Launch jobs

In [40]:
def zne_job_launch(tomo_circs, backend, optimization_level, shots):

    # This function runs the tomography circuits and unrolls the gates to increase the noise level
    # The counts that are obtained for the differnt noise levels are then extrapolated to the zero-noise level

    zne_result_list = []
    scale_factors = [1.0, 2.0, 3.0]


    # Unfold the tomography circuit by a scale factor and evaluate them 
    noise_scaled_circuits = [[zne.scaling.fold_global(circ, s) for s in scale_factors] for circ in tomo_circs] 
    noise_scaled_circuits = list(chain(*noise_scaled_circuits)) 

    
    job    = execute(noise_scaled_circuits, backend=backend, optimization_level=optimization_level, shots=shots)
    job_id = job.job_id()
    ## Now we will save these results in a file, together with the circuits that have generated them
    
    pickle_file = "./hw_data/job_"+str(job_id)+"_circuits"
    pickle_data = {}
    pickle_data["tomo_circs"]            = tomo_circs
    pickle_data["scale_factors"]         = scale_factors
    pickle_data["noise_scaled_circuits"] = noise_scaled_circuits
    
    # Dump on file
    with open(pickle_file,'wb+') as f:
        pickle.dump(pickle_data, f)
    
    print("JOB "+str(job_id)+" SUBMITTED")
    return str(job_id), job

In [41]:
# Create the tomography circuits
st_qcs = state_tomography_circuits(fqc.decompose(), [fqr[1], fqr[3], fqr[5]])
#st_qcs = state_tomography_circuits(fqc.decompose(), [fqr[0], fqr[1], fqr[2]])

In [42]:
# Repeat fidelity measurement
reps = 1 # Needs to be 8 in the final execution
job_list = [] # We don't actually need it...
job_id_list = []

job_file = "submitted_jobs.txt"

## This part removes the file in order to not mix different launches (e.g. simulator and hw)
if os.path.exists(job_file):
    os.remove(job_file)
    print("Previous file deleted")
else:
    print("Can not delete the file as it doesn't exists")

    

# Now launch the jobs
for count in range(reps):
    print("\n REPETITION "+str(count+1)+"\n")
    
    zne_job_id, zne_job = zne_job_launch(st_qcs, backend=backend, optimization_level=0, shots=shots)
    
    with open(job_file,"a") as f:
        f.write(zne_job_id)
        f.write("\n")

    job_list.append(zne_job)
    job_id_list.append(zne_job_id)

Previous file deleted

 REPETITION 1

JOB 623dd6930af65d0ea1d938c5 SUBMITTED


In [44]:
job_id_list

['623dd6930af65d0ea1d938c5']

### Collect Jobs

In [45]:
def zne_results_collect(backend,jobid, zne_order, shots):
    
    job    = backend.retrieve_job(jobid) # Maybe a [0] is needed if it doesn't work properly
    result = job.result()
    
    filename = "./hw_data/job_"+str(jobid)+"circuits"
    result_file = pickle.load(open(filename,'rb'))
    print("\nLOADED FILE: "+filename)
    
    tomo_circs = result_file["tomo_circs"]
    
    count_list = result.get_counts()
    ordered_bitstrings = dict(sorted(count_list[0].items()))
    
    zne_result_list = []
    scale_factors = [1.0, 2.0, 3.0]

    for i in range(len(tomo_circs)):
        counts_dict = {}

        # Loop over the results of the scaled circuits and collect the data in the correct form
        for key in ordered_bitstrings.keys():
            counts_list_zne = []
            for count in count_list[i*len(scale_factors):len(scale_factors)*(i+1)]:
                counts_list_zne.append(count[key])
            # Here we extrapolate the counts to zero noise and round to the closest integer 
            zne_counts_value = int(zne.PolyFactory.extrapolate(scale_factors, counts_list_zne, order=zne_order)) 
            if zne_counts_value < 0:
                zne_counts_value = 0
            counts_dict[key] = zne_counts_value
        zne_result_list.append(counts_dict)
        
    # To work with the StateTomographyFitter we need to put the result into a Qiskit Result() object
    name_list = [circ.name for circ in tomo_circs]
    results_tmp = [[ExperimentResult(shots=shots, success=True, data=ExperimentResultData(counts=result_i), header=QobjExperimentHeader(name=name_i))] for (name_i, result_i) in zip(name_list, zne_result_list)]
    results = [Result(backend_name="zne", backend_version="zne", qobj_id='0', job_id='0', success=True, results=result_i) for result_i in results_tmp]

    return results, tomo_circs

In [46]:
## Here we put the job_id_list created

job_id_list = ['623dd6930af65d0ea1d938c5']

In [48]:
#backend = jakarta
ibmq_fids     = []

for job in job_id_list:
    #print(job)
    
    job_file = "./hw_data/tomography_id_"+job
    zne_res, zne_circs = zne_results_collect(backend=backend,jobid=job, zne_order=2, shots=shots)
    ibmq_fids.append(state_tomo(zne_res, zne_circs))


LOADED FILE: ./hw_data/job_623dd6930af65d0ea1d938c5circuits


In [49]:
## Print the final result
print('state tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean(ibmq_fids), np.std(ibmq_fids))) 

state tomography fidelity = 0.6058 ± 0.0000
